In [18]:
user_mame = input("Username")
user_password = input("Password")

In [6]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pandas as pd
import os
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [7]:
current_path = os.getcwd()
current_path = current_path.split('Beer_Recommendation_Engine')
path = current_path[0] + 'Beer_Recommendation_Engine'
path

'/Users/drknt_rises/Desktop/Personal/Git Repos/Beer_Recommendation_Engine'

In [8]:
def login():
    wd = webdriver.Chrome(options=chrome_options)

    url = 'https://www.beeradvocate.com/community/login/'

    wd.get(url)
    username = WebDriverWait(wd, 20).until(
                    EC.element_to_be_clickable(
                        wd.find_element(by=By.XPATH, value='/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[2]/div/form/dl[1]/dd/input')))
    username.send_keys(user_mame)

    password = WebDriverWait(wd, 20).until(
                    EC.element_to_be_clickable(
                        wd.find_element(by=By.XPATH, value='/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[2]/div/form/dl[2]/dd/ul/li[3]/input')))
    password.send_keys(user_password)

    remeber_password = WebDriverWait(wd, 20).until(
                    EC.element_to_be_clickable(
                        wd.find_element(by=By.XPATH, value='/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[2]/div/form/dl[3]/dd/label/input')))
    remeber_password.click()

    login = WebDriverWait(wd, 20).until(
                    EC.element_to_be_clickable(
                        wd.find_element(by=By.XPATH, value='/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[2]/div/form/dl[3]/dd/input')))
    login.click()

    return wd

In [9]:
wd = webdriver.Chrome(options=chrome_options)

wd = login()

url = 'https://www.beeradvocate.com/beer/top-rated/'

wd.get(url)

table = wd.find_element(by=By.TAG_NAME, value='table')
brands = table.find_elements(by=By.TAG_NAME, value='a')

beer_brand_sites = []
i = 1
for brand in tqdm(brands):
    brand_url = brand.get_attribute('href')
    name = brand.text
    len_url = len(brand_url.split('/'))
    if len_url == 8:
        beer_brand_sites.append([name, brand_url, i])
        i += 1

beer_brand_sites = pd.DataFrame(beer_brand_sites, columns=['beer_name', 'url', 'pos'])
beer_brand_sites

100%|██████████| 750/750 [00:07<00:00, 106.37it/s]


,beer_name,url,pos
0,Kentucky Brunch Brand Stout,https://www.beeradvocate.com/beer/profile/2322...,1
1,Vanilla Bean Assassin,https://www.beeradvocate.com/beer/profile/2322...,2
2,Marshmallow Handjee,https://www.beeradvocate.com/beer/profile/26/4...,3
3,Abraxas - Barrel-Aged,https://www.beeradvocate.com/beer/profile/2588...,4
4,Hunahpu's Imperial Stout - Double Barrel Aged,https://www.beeradvocate.com/beer/profile/1798...,5
...,...,...,...
245,The Abyss,https://www.beeradvocate.com/beer/profile/63/3...,246
246,All That Is And All That Ever Will Be,https://www.beeradvocate.com/beer/profile/2874...,247
247,Abraxas,https://www.beeradvocate.com/beer/profile/2588...,248
248,Stickee Monkee,https://www.beeradvocate.com/beer/profile/2210...,249


In [17]:
pos = 140
for url in (pbar := tqdm(beer_brand_sites['url'][pos:])):
    scraped_data = []
    url = url + '?view=beer&show=recent&start=' + str(0) + '#lists'
    wd.get(url)
    links = wd.find_elements(by=By.TAG_NAME, value='a')

    l = []
    for link in links:
        url = link.get_attribute('href')
        if url != None:
            if 'recent&start' in url:
                url = url.split('?view=beer&show=recent&start=')[-1]
                last = url.split('#')[0]
                l.append(int(last))
    l = max(l)
    
    i = 1
    url = beer_brand_sites['url'][pos]
    while i<=l:
        if i == 1:
            i = 0
            beer_url = url + '?view=beer&show=recent&start=' + str(i) + '#lists'
            i = 20
        else:
            beer_url = url + '?view=beer&show=recent&start=' + str(i) + '#lists'
            i += 20
            
        pbar.set_description(f"Fetching data from {beer_url}")

        wd.get(beer_url)

        comms = wd.find_elements(by=By.ID, value='rating_fullview_container')

        for comm in comms:
            row = [beer_brand_sites['pos'][pos], beer_brand_sites['beer_name'][pos]]
            data = comm.find_element(by=By.ID, value='rating_fullview_content_2')
            data = data.text
            data = data.split('\n')
            # print(data)
            if len(data) == 3:
                rating = data[0].split('Rated: ')
                rating = rating[1].split(' ')
                rating = float(rating[0])
                row.append(rating)
                
                for j in range(5):
                    row.append(np.nan)

                user = data[0].split('by ')
                user = user[1].split(' ')
                user = str(user[0])
                row.append(user)

                place = data[0].split('from ')
                place = str(place[-1])
                row.append(place)

                row.append(None)

                date = data[-1].split(' ')
                if str(date[-1]) == 'Report':
                    date = " ".join(date[0:-1])
                    date = date.strip()
                else:
                    date = data[-1]    
                row.append(date)
            elif str(data[3])[:4] == 'look':
                rating = data[2].split('/')
                rating = float(rating[0])
                row.append(rating)

                attributes = data[3].split('|')
                for attribute in attributes:
                    attribute = attribute.strip()
                    attribute = float(attribute.split(' ')[1])
                    row.append(attribute)

                user = data[0].split('by ')
                user = user[1].split(' ')
                user = str(user[0])
                row.append(user)

                place = data[0].split('from ')
                place = str(place[-1])
                row.append(place)

                comment = " ".join(data[4:-1])
                comment = comment.strip()
                row.append(comment)

                date = data[-1].split(' ')
                if str(date[-1]) == 'Report':
                    date = " ".join(date[0:-1])
                    date = date.strip()
                else:
                    date = data[-1]    
                row.append(date)
            else:
                rating = data[2].split('/')
                rating = float(rating[0])
                row.append(rating)

                for j in range(5):
                    row.append(np.nan)

                user = data[0].split('by ')
                user = user[1].split(' ')
                user = str(user[0])
                row.append(user)

                place = data[0].split('from ')
                place = str(place[-1])
                row.append(place)

                comment = " ".join(data[3:-1])
                comment = comment.strip()
                row.append(comment)

                date = data[-1].split(' ')
                if str(date[-1]) == 'Report':
                    date = " ".join(date[0:-1])
                    date = date.strip()
                else:
                    date = data[-1]    
                row.append(date)
            scraped_data.append(row)
    
    scraped_data = pd.DataFrame(scraped_data, columns=['pos', 'beer_name', 'final_rating', 'looks', 'smell', 'taste', 'feel', 'overall', 'user', 
                                                   'place', 'comment', 'date'])
    if pos==0:
        scraped_data.to_csv(path + '/Data/beer_reviews.csv', mode='a', index=False, header=True)
    else:
        scraped_data.to_csv(path + '/Data/beer_reviews.csv', mode='a', index=False, header=False)
    pos += 1

wd.quit()

Fetching data from https://www.beeradvocate.com/beer/profile/35/25759/?view=beer&show=recent&start=2800#lists: 100%|██████████| 110/110 [3:05:24<00:00, 101.13s/it]     


In [15]:
scraped_data = pd.read_csv(path + '/Data/beer_reviews.csv')
scraped_data

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,comment,date
0,1,Kentucky Brunch Brand Stout,4.75,NaN,NaN,NaN,NaN,NaN,ryancl,Minnesota,NaN,"Aug 27, 2023"
1,1,Kentucky Brunch Brand Stout,5.00,5.0,5.00,5.0,5.00,5.00,poidog,Hawaii,"Amazing stout, expensive but worth the price!","Aug 06, 2023"
2,1,Kentucky Brunch Brand Stout,4.52,NaN,NaN,NaN,NaN,NaN,notchucknorris,California,NaN,"Jul 26, 2023"
3,1,Kentucky Brunch Brand Stout,5.00,NaN,NaN,NaN,NaN,NaN,CraftFan5,New Jersey,NaN,"Jul 21, 2023"
4,1,Kentucky Brunch Brand Stout,5.00,5.0,5.00,5.0,5.00,5.00,Filabee,Michigan,This beer meet and exceeded all the unreal hyp...,"Jul 08, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...
259241,139,Modem Tones - Bourbon Barrel-Aged - Vanilla,4.78,NaN,NaN,NaN,NaN,NaN,kylehay2004,Illinois,NaN,"Feb 10, 2018"
259242,139,Modem Tones - Bourbon Barrel-Aged - Vanilla,4.57,NaN,NaN,NaN,NaN,NaN,johnInLA,California,NaN,"Feb 08, 2018"
259243,139,Modem Tones - Bourbon Barrel-Aged - Vanilla,4.98,NaN,NaN,NaN,NaN,NaN,tonystarks,California,NaN,"Feb 07, 2018"
259244,139,Modem Tones - Bourbon Barrel-Aged - Vanilla,4.87,5.0,4.75,5.0,4.75,4.75,bobstockwell,Massachusetts,"Had at modern times Dtla . Great taste, thick ...","Jan 27, 2018"
